In [36]:
import pandas as pd 
import matplotlib.pyplot as plt 


In [91]:
df = pd.read_csv("E:\\MACHINE LEARNING\\datasets\\Titanic-Dataset.csv")
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


1.  handaling the null values 

In [92]:

df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [93]:
from sklearn.impute import KNNImputer,SimpleImputer

In [94]:
# KNNImputer apply for age columns 
impute = KNNImputer()
imputed = impute.fit_transform(df[['Age']])
age_df = pd.DataFrame(imputed,columns=['age'])
age_df.head()


,age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0


In [95]:
 # SimpleImputer apply for embark columns 
sim_impute = SimpleImputer(strategy='most_frequent')
sim_imputed = sim_impute.fit_transform(df[['Embarked']])
embark_df = pd.DataFrame(sim_imputed,columns=['embarked'])
embark_df.head()

,embarked
0,S
1,C
2,S
3,S
4,S


In [100]:
df2 = df[['Sex','Fare','Pclass','Survived']]

In [101]:
x = pd.concat([embark_df,age_df,df2],ignore_index=True,axis=1)
x.columns=['embark','sex','fare','class','age','Survived']
x.head()

,embark,sex,fare,class,age,Survived
0,S,22.0,male,7.2500,3,0
1,C,38.0,female,71.2833,1,1
2,S,26.0,female,7.9250,3,1
3,S,35.0,female,53.1000,1,1
4,S,35.0,male,8.0500,3,0
